# **NexFin**

A multi-agent collaboration for Financial Analysis

- Autonomous work flow

### **Install Required Packages**

In [1]:
%pip install --quiet -U crewai
%pip install --quiet -U crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### **Import Required Libraries**

In [3]:
from crewai import Agent, Task, Crew, LLM
from google.colab import userdata
import os
from IPython.display import Markdown
from IPython.display import  display
import textwrap

### **Set Up API Key**

In [4]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

### **Initialize an LLM**

In [5]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=1.0,
)

### **Define Tools**

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### **Create Agents**

Define Agents, and provide them a `role`, `goal` and `backstory`.

In [7]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

In [8]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

In [9]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

In [10]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

### **Create Tasks**

In [12]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [13]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [14]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [15]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

### **Creating the Crew**

In [16]:
from crewai import Process

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, trading_strategy_agent, execution_agent, risk_management_agent],
    tasks=[data_analysis_task, strategy_development_task, execution_planning_task, risk_assessment_task],
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

### **Running the Crew and Displaying Response**

In [17]:
def to_markdown(text):
    # Replace bullet points with asterisks for markdown formatting
    text = text.replace("•", "  *")

    # Indent the text with a '>' character for blockquote style
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [18]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [19]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.
 

I encountered an error while trying to use the tool. This was the error: unhashable type: 'dict'.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description': 'The context for the task', 'type': 'str'}, 'coworker': {'description': 'The role/name of the coworker to delegate to', 'type': 'str'}}
Tool Description: Delegate a specific task to one of the following coworkers: Data Analyst
The input to this tool should be the coworker, the task you want them to do, and ALL necessary context to execute the task, they know nothing about the task, so share absolute everything you know, don't reference things but instead explain them.



# Agent: Crew Manager


In [20]:
to_markdown(result.raw)

> **Risk Analysis Report: Proposed AAPL Trading Strategies**
> 
> **Introduction:**
> 
> This report analyzes the risks associated with three proposed AAPL day trading strategies: an LSTM-based momentum strategy, a mean reversion strategy using VIX and SP500 data, and a combined LSTM and mean reversion strategy.  The analysis assumes a medium risk tolerance.  This analysis is based on general knowledge of these trading strategies and does not include specifics regarding parameters or implementation. Therefore, the analysis provided is a high-level overview and should not be considered a comprehensive risk assessment of your specific strategies.  Thorough backtesting and parameter optimization are crucial before live trading.
> 
> **Strategy 1: LSTM-based Momentum Trading**
> 
> **Risks:**
> 
> * **Overfitting:** LSTM models can overfit to historical data, leading to poor performance on new data.  This risk is mitigated through robust cross-validation techniques and regularization.
> * **Data Quality:** The accuracy of the model depends heavily on the quality and completeness of the training data. Noise or biases in the data can lead to inaccurate predictions. Data cleansing and validation are critical.
> * **Market Regime Shifts:** Momentum strategies may perform poorly during market regime shifts or periods of high volatility. Risk mitigation requires incorporating volatility indicators and adjusting position sizing accordingly.
> * **Transaction Costs:**  Frequent trading inherent in momentum strategies amplifies the impact of transaction costs.  These must be carefully factored into performance calculations.
> 
> 
> **Mitigation Strategies:**
> 
> * Rigorous backtesting with out-of-sample data.
> * Regular model retraining and validation.
> * Incorporating volatility measures (e.g., Bollinger Bands, Average True Range) to dynamically adjust position sizing.
> * Careful consideration of transaction costs in profitability assessment.
> 
> **Strategy 2: Mean Reversion Strategy using VIX and SP500**
> 
> **Risks:**
> 
> * **False Signals:** Mean reversion strategies can generate false signals during prolonged trends. This risk is mitigated through careful parameter selection and the use of filters.
> * **Volatility Clustering:** Periods of high volatility can disrupt the mean reversion process.  Strategies should incorporate measures to manage risk during these periods.
> * **Market Regime Shifts:** Mean reversion strategies may fail during prolonged trends.  Risk mitigation involves incorporating trend indicators and dynamically adjusting the strategy.
> * **Correlation Breakdown:**  The relationship between AAPL, VIX, and SP500 may change over time, reducing the effectiveness of the mean reversion strategy.  Continuous monitoring of correlations is essential.
> 
> **Mitigation Strategies:**
> 
> * Thorough backtesting under various market conditions.
> * Using robust statistical methods for identifying mean reversion opportunities.
> * Incorporating trend-following indicators as filters.
> * Dynamically adjusting position sizing based on volatility.
> * Continuously monitoring correlations between AAPL, VIX, and SP500.
> 
> **Strategy 3: Combined LSTM and Mean Reversion Strategy**
> 
> **Risks:**
> 
> This strategy combines the risks of both individual strategies.  The complexity increases the difficulty of identifying and mitigating potential issues. The interaction between the LSTM model's predictions and the mean reversion signals needs careful analysis.
> 
> 
> **Mitigation Strategies:**
> 
> * Extensive backtesting and stress testing under various market conditions.
> * Careful parameter tuning for both components of the strategy.
> * Incorporating risk management techniques from both strategies.
> * Continuous monitoring and adjustment of the strategy based on performance and market dynamics.
> 
> 
> **Overall Mitigation Strategies:**
> 
> * **Position Sizing:**  Implement a robust position sizing methodology to limit potential losses on any single trade.
> * **Stop-Loss Orders:** Employ stop-loss orders to automatically exit positions when losses reach a predetermined level.
> * **Risk Monitoring:** Regularly monitor the performance of the strategies and make adjustments as needed.
> * **Diversification:** Consider diversifying across different asset classes to reduce overall portfolio risk.
> * **Stress Testing:**  Perform stress tests to assess the strategy's resilience to extreme market events.
> 
> 
> 
> **Disclaimer:** This risk analysis is for informational purposes only and should not be considered investment advice. The information provided is not exhaustive and does not cover all potential risks.  The actual performance of these strategies may vary significantly from the results of backtesting.  It is crucial to conduct thorough due diligence and consult with a financial professional before implementing any trading strategy.

---